In [ ]:
import torch
from prep_data import download_high_quality

In [ ]:
# Download high-quality Polish datasets (once)
download_high_quality("1000_novels_corpus_CLARIN-PL", "data/prose_train.txt")
download_high_quality("wolne_lektury_corpus", "data/lektury_val.txt")